In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\bin\\spark-3.3.0-bin-hadoop2'

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf=SparkConf().setAppName("practice").setMaster("local")
sc=SparkContext(conf=conf)

In [4]:
from pyspark.sql import SparkSession 
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
# import pyspark.mllib.linalg

In [6]:
spark = SparkSession.builder.appName("Pyspark ML algrithms").getOrCreate()

In [7]:
data = spark.read.csv('Admission_Prediction.csv',header=True)

In [8]:
data.show()

+----------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+----------+---------+-----------+-----------------+---+---+----+--------+---------------+
|         1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|         2|      324|        107|                4|  4|4.5|8.87|       1|           0.76|
|         3|     null|        104|                3|  3|3.5|   8|       1|           0.72|
|         4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|         5|      314|        103|                2|  2|  3|8.21|       0|           0.65|
|         6|      330|        115|                5|4.5|  3|9.34|       1|            0.9|
|         7|      321|        109|             null|  3|  4| 8.2|       1|           0.75|
|         8|      308|        101|                2|  3|  4| 7.9|       0|           0.68|

In [9]:
data.printSchema()

root
 |-- Serial No.: string (nullable = true)
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [10]:
type(data)

pyspark.sql.dataframe.DataFrame

In [11]:
data.columns

['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit']

In [12]:
data = data.drop("Serial No.")

In [13]:
data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|  4|4.5|8.87|       1|           0.76|
|     null|        104|                3|  3|3.5|   8|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|  2|  3|8.21|       0|           0.65|
|      330|        115|                5|4.5|  3|9.34|       1|            0.9|
|      321|        109|             null|  3|  4| 8.2|       1|           0.75|
|      308|        101|                2|  3|  4| 7.9|       0|           0.68|
|      302|        102|                1|  2|1.5|   8|       0|            0.5|
|      323|        108|                3

In [14]:
data.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research',
 'Chance of Admit']

In [15]:
from pyspark.sql.functions import col

for c in data.columns:
    print(col(c))

Column<'GRE Score'>
Column<'TOEFL Score'>
Column<'University Rating'>
Column<'SOP'>
Column<'LOR'>
Column<'CGPA'>
Column<'Research'>
Column<'Chance of Admit'>


In [16]:
data.select(*(col(c) for c in data.columns)).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|  4|4.5|8.87|       1|           0.76|
|     null|        104|                3|  3|3.5|   8|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|  2|  3|8.21|       0|           0.65|
|      330|        115|                5|4.5|  3|9.34|       1|            0.9|
|      321|        109|             null|  3|  4| 8.2|       1|           0.75|
|      308|        101|                2|  3|  4| 7.9|       0|           0.68|
|      302|        102|                1|  2|1.5|   8|       0|            0.5|
|      323|        108|                3

In [17]:
new_data = data.select(*(col(c).cast("float") for c in data.columns))
new_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|
|     null|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|
|    321.0|      109.0|             null|3.0|4.0| 8.2|     1.0|           0.75|
|    308.0|      101.0|              2.0|3.0|4.0| 7.9|     0.0|           0.68|
|    302.0|      102.0|              1.0|2.0|1.5| 8.0|     0.0|            0.5|
|    323.0|      108.0|              3.0

In [18]:
new_data.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [19]:
from pyspark.sql.functions import col, count, isnan, when

In [20]:
new_data.select([count(when(col(c).isNull(), c)) for c in new_data.columns]).show()

+-------------------------------------------------------+-----------------------------------------------------------+-----------------------------------------------------------------------+-------------------------------------------+-------------------------------------------+---------------------------------------------+-----------------------------------------------------+-------------------------------------------------------------------+
|count(CASE WHEN (GRE Score IS NULL) THEN GRE Score END)|count(CASE WHEN (TOEFL Score IS NULL) THEN TOEFL Score END)|count(CASE WHEN (University Rating IS NULL) THEN University Rating END)|count(CASE WHEN (SOP IS NULL) THEN SOP END)|count(CASE WHEN (LOR IS NULL) THEN LOR END)|count(CASE WHEN (CGPA IS NULL) THEN CGPA END)|count(CASE WHEN (Research IS NULL) THEN Research END)|count(CASE WHEN (Chance of Admit IS NULL) THEN Chance of Admit END)|
+-------------------------------------------------------+---------------------------------------------------

In [21]:
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [23]:
from pyspark.ml.feature import Imputer

In [24]:
imputer = Imputer(inputCols=["GRE Score","TOEFL Score","University Rating"],
                 outputCols=["GRE Score","TOEFL Score","University Rating"])

model = imputer.fit(new_data)

imputed_data = model.transform(new_data)

In [25]:
imputed_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|
|316.55878|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|
|    321.0|      109.0|        3.1216495|3.0|4.0| 8.2|     1.0|           0.75|
|    308.0|      101.0|              2.0|3.0|4.0| 7.9|     0.0|           0.68|
|    302.0|      102.0|              1.0|2.0|1.5| 8.0|     0.0|            0.5|
|    323.0|      108.0|              3.0

In [26]:
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [27]:
features = imputed_data.drop('Chance of Admit')

In [28]:
features.columns

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR',
 'CGPA',
 'Research']

In [30]:
assembler =  VectorAssembler( inputCols=features.columns, outputCol='features')

In [31]:
output = assembler.transform(imputed_data)

In [32]:
output.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|            features|
+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|[337.0,118.0,4.0,...|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|[324.0,107.0,4.0,...|
|316.55878|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|[316.558776855468...|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|[322.0,110.0,3.0,...|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|[314.0,103.0,2.0,...|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|[330.0,115.0,5.0,...|
|    321.0|      109.0|        3.1216495|3.0|4.0| 8.2|     1.0|           0.75|[321.0,109.0

In [33]:
data2 = output.select("features","Chance of Admit")

In [34]:
data2.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[337.0,118.0,4.0,...|           0.92|
|[324.0,107.0,4.0,...|           0.76|
|[316.558776855468...|           0.72|
|[322.0,110.0,3.0,...|            0.8|
|[314.0,103.0,2.0,...|           0.65|
|[330.0,115.0,5.0,...|            0.9|
|[321.0,109.0,3.12...|           0.75|
|[308.0,101.0,2.0,...|           0.68|
|[302.0,102.0,1.0,...|            0.5|
|[323.0,108.0,3.0,...|           0.45|
|[325.0,106.0,3.0,...|           0.52|
|[327.0,111.0,4.0,...|           0.84|
|[316.558776855468...|           0.78|
|[307.0,109.0,3.0,...|           0.62|
|[311.0,104.0,3.0,...|           0.61|
|[314.0,105.0,3.0,...|           0.54|
|[317.0,107.0,3.0,...|           0.66|
|[319.0,106.0,3.0,...|           0.65|
|[318.0,110.0,3.0,...|           0.63|
|[303.0,102.0,3.0,...|           0.62|
+--------------------+---------------+
only showing top 20 rows



In [35]:
output.select("features").toPandas().values

array([[DenseVector([337.0, 118.0, 4.0, 4.5, 4.5, 9.65, 1.0])],
       [DenseVector([324.0, 107.0, 4.0, 4.0, 4.5, 8.87, 1.0])],
       [DenseVector([316.5588, 104.0, 3.0, 3.0, 3.5, 8.0, 1.0])],
       [DenseVector([322.0, 110.0, 3.0, 3.5, 2.5, 8.67, 1.0])],
       [DenseVector([314.0, 103.0, 2.0, 2.0, 3.0, 8.21, 0.0])],
       [DenseVector([330.0, 115.0, 5.0, 4.5, 3.0, 9.34, 1.0])],
       [DenseVector([321.0, 109.0, 3.1216, 3.0, 4.0, 8.2, 1.0])],
       [DenseVector([308.0, 101.0, 2.0, 3.0, 4.0, 7.9, 0.0])],
       [DenseVector([302.0, 102.0, 1.0, 2.0, 1.5, 8.0, 0.0])],
       [DenseVector([323.0, 108.0, 3.0, 3.5, 3.0, 8.6, 0.0])],
       [DenseVector([325.0, 106.0, 3.0, 3.5, 4.0, 8.4, 1.0])],
       [DenseVector([327.0, 111.0, 4.0, 4.0, 4.5, 9.0, 1.0])],
       [DenseVector([316.5588, 112.0, 4.0, 4.0, 4.5, 9.1, 1.0])],
       [DenseVector([307.0, 109.0, 3.0, 4.0, 3.0, 8.0, 1.0])],
       [DenseVector([311.0, 104.0, 3.0, 3.5, 2.0, 8.2, 1.0])],
       [DenseVector([314.0, 105.0, 3.0, 3

In [ ]:
#train test split

In [36]:
train_df,test_df = data2.randomSplit([0.75,0.25])

In [37]:
train_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,93.0,1.0,1...|           0.46|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,93.0,1.0,2...|           0.46|
|[295.0,96.0,2.0,1...|           0.47|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,99.0,2.0,2...|           0.57|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,99.0,2.0,2...|           0.61|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,96.0,2.0,2...|           0.34|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,97.0,3.121...|           0.45|
|[298.0,98.0,2.0,4...|           0.34|
+--------------------+---------------+
only showing top 20 rows



In [38]:
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[295.0,101.0,2.0,...|           0.69|
|[296.0,97.0,2.0,1...|           0.49|
|[297.0,98.0,2.0,2...|           0.59|
|[298.0,92.0,1.0,2...|           0.51|
|[298.0,98.0,2.0,1...|           0.44|
|[298.0,101.0,4.0,...|           0.53|
|[299.0,97.0,3.0,5...|           0.38|
|[299.0,100.0,3.0,...|           0.42|
|[300.0,99.0,1.0,1...|           0.58|
|[300.0,100.0,3.0,...|           0.64|
|[300.0,105.0,1.0,...|           0.58|
|[301.0,96.0,1.0,3...|           0.54|
|[301.0,98.0,1.0,2...|           0.67|
|[301.0,104.0,3.0,...|           0.68|
|[301.0,107.0,3.0,...|           0.62|
|[302.0,99.0,1.0,2...|           0.57|
|[302.0,99.0,3.0,2...|           0.52|
|[302.0,102.0,1.0,...|            0.5|
|[302.0,102.0,3.0,...|           0.65|
|[303.0,99.0,3.0,2...|           0.36|
+--------------------+---------------+
only showing top 20 rows



# Linear Regressor

In [39]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit')
linear_model = lin_reg.fit(train_df)

In [40]:
linear_model.coefficients

DenseVector([0.0016, 0.0022, 0.009, -0.0004, 0.018, 0.1246, 0.0279])

In [41]:
linear_model.intercept

-1.1907969535121155

In [42]:
trainSummary = linear_model.summary

print("RMSE :", trainSummary.rootMeanSquaredError)
print("r2 Score :", trainSummary.r2)

RMSE : 0.06024156262009677
r2 Score : 0.8189122283914023


In [43]:
## Prediction

predictions = linear_model.transform(test_df)
predictions.select("Prediction","Chance of Admit", "features").show()

+-------------------+---------------+--------------------+
|         Prediction|Chance of Admit|            features|
+-------------------+---------------+--------------------+
| 0.5312469084876466|           0.69|[295.0,101.0,2.0,...|
| 0.5170165712330894|           0.49|[296.0,97.0,2.0,1...|
| 0.5221813468576291|           0.59|[297.0,98.0,2.0,2...|
|  0.509952554616895|           0.51|[298.0,92.0,1.0,2...|
| 0.5219221057912249|           0.44|[298.0,98.0,2.0,1...|
| 0.6058437590935941|           0.53|[298.0,101.0,4.0,...|
|  0.538915617381263|           0.38|[299.0,97.0,3.0,5...|
| 0.5078430972909838|           0.42|[299.0,100.0,3.0,...|
| 0.5540829527695599|           0.58|[300.0,99.0,1.0,1...|
| 0.6918478860681361|           0.64|[300.0,100.0,3.0,...|
| 0.5320593356319736|           0.58|[300.0,105.0,1.0,...|
| 0.5191957371431426|           0.54|[301.0,96.0,1.0,3...|
| 0.5924701297732111|           0.67|[301.0,98.0,1.0,2...|
| 0.6522896002347434|           0.68|[301.0,104.0,3.0,..

In [44]:
from pyspark.ml.evaluation import RegressionEvaluator

In [45]:
pred_evaluator = RegressionEvaluator(predictionCol ="prediction", labelCol="Chance of Admit", metricName="r2")
print("R2 on test data =", pred_evaluator.evaluate(predictions))

R2 on test data = 0.8227140954724383


# Random Forest Regressor

In [46]:
random_forest_rg = RandomForestRegressor(featuresCol = 'features', labelCol='Chance of Admit')

In [47]:
#train model
model = random_forest_rg.fit(train_df)

In [48]:
#predictions

predictions = model.transform(test_df)

In [49]:
predictions.show()

+--------------------+---------------+-------------------+
|            features|Chance of Admit|         prediction|
+--------------------+---------------+-------------------+
|[295.0,101.0,2.0,...|           0.69| 0.5102304870371737|
|[296.0,97.0,2.0,1...|           0.49| 0.5095086724169315|
|[297.0,98.0,2.0,2...|           0.59| 0.5343462711085416|
|[298.0,92.0,1.0,2...|           0.51| 0.4941349109419163|
|[298.0,98.0,2.0,1...|           0.44|0.48644817513377625|
|[298.0,101.0,4.0,...|           0.53| 0.5668283153046231|
|[299.0,97.0,3.0,5...|           0.38| 0.5314790076087842|
|[299.0,100.0,3.0,...|           0.42| 0.5066308024991197|
|[300.0,99.0,1.0,1...|           0.58| 0.5647684339130161|
|[300.0,100.0,3.0,...|           0.64| 0.6853956046211199|
|[300.0,105.0,1.0,...|           0.58| 0.5185965090821657|
|[301.0,96.0,1.0,3...|           0.54| 0.5309556694814479|
|[301.0,98.0,1.0,2...|           0.67| 0.6199714426525726|
|[301.0,104.0,3.0,...|           0.68| 0.660157350834520

In [50]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit",predictionCol ="prediction", metricName="rmse")
print("RMSE on test data =", evaluator.evaluate(predictions))

RMSE on test data = 0.059505025442748084


In [51]:
evaluator = RegressionEvaluator(labelCol="Chance of Admit",predictionCol ="prediction", metricName="r2")
print("R2 on test data =", evaluator.evaluate(predictions))

R2 on test data = 0.8172290422746636
